Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler , StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

Read data 

In [2]:
#import pandas as pd
data=pd.read_csv("Random Forest results(Date Index).CSV" )
data['Date'] = pd.to_datetime(data['Date'])
dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)
#dataset.head(2)
#data

<ipython-input-2-56bfd69fc925>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)


Add year & month column

✈ 🏸 Feature engineering

In [3]:
#['year'] = pd.DatetimeIndex(df2['Date']).year
data['month'] = pd.DatetimeIndex(data['Date']).month
data['day'] = pd.DatetimeIndex(data['Date']).day
data.head(2)

,Date,Load,LBMP,DI,WC,wetbulb,T2MWET,winddirection,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_UVA,ALLSKY_SFC_UV_INDEX,specifichumidity,PS,month,day
0,2016-01-01 00:00:00,4894.5,213.20,37.974654,2.616960,1.30,2.00,300.90,0.0,0.0,0.0,4.15,101.28,1,1
1,2016-01-01 01:00:00,4712.7,19.08,37.706599,2.621161,1.01,1.76,298.94,0.0,0.0,0.0,4.03,101.27,1,1


onehot_encode_integers & nomrlize load data

✈ 🏸 Special one hot encoding for time-series data

✈ 🏸 MinMaxScalar

In [4]:
def select_columns(df):
  cols_to_keep = [       
      'Load',
      'LBMP',
      'month',
      'day',

  ]
  df_subset = df[cols_to_keep]
  return df_subset

def onehot_encode_integers(df, excluded_cols):
  df = df.copy()

  int_cols = [col for col in df.select_dtypes(include=['int'])
              if col not in excluded_cols]

  df.loc[:,int_cols] = df.loc[:,int_cols].astype('str')

  df_encoded = pd.get_dummies(df)
  return df_encoded

def normalize_cnt(df):
    df = df.copy()
    df['Load'] = df['Load'] / df['Load'].max()
    df['LBMP'] = df['LBMP'] / df['LBMP'].max()

    return df

In [5]:
mydata = (data
           .pipe(select_columns)
           .pipe(onehot_encode_integers, 
                 excluded_cols=['cnt'])
           .pipe(normalize_cnt)
)

In [6]:
mydata.head(2)

,Load,LBMP,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,...,day_29,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9
0,0.442153,0.115957,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.425730,0.010377,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def create_dataset(df, n_deterministic_features, 
                   window_size, forecast_size, 
                   batch_size):
  # Feel free to play with shuffle buffer size
  shuffle_buffer_size = len(df)
  # Total size of window is given by the number of steps to be considered
  # before prediction time + steps that we want to forecast
  total_size = window_size + forecast_size

  data = tf.data.Dataset.from_tensor_slices(df.values)

  # Selecting windows
  data = data.window(total_size, shift=1, drop_remainder=True)
  data = data.flat_map(lambda k: k.batch(total_size))

  # Shuffling data (seed=Answer to the Ultimate Question of Life, the Universe, and Everything)
  data = data.shuffle(shuffle_buffer_size, seed=42)

  # Extracting past features + deterministic future + labels
  data = data.map(lambda k: ((k[:-forecast_size],
                             k[-forecast_size:,-n_deterministic_features:]),
                             k[-forecast_size:,0]))
  
  return data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
mydata

,Load,LBMP,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,...,day_29,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9
0,0.442153,0.115957,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.425730,0.010377,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.407455,0.013303,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.394627,0.006195,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.387255,0.001077,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,0.560078,0.013238,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
35036,0.541758,0.011204,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
35037,0.523366,0.011617,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
35038,0.497322,0.011655,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
len(mydata.columns) 

45

In [10]:
# Times at which to split train/validation and validation/test
val_time = 24500
test_time = 29750

# How much data from the past should we need for a forecast?
window_len = 24 # 24-hour
# How far ahead do we want to generate forecasts?
forecast_len = 1 # one-hour ahead

# Auxiliary constants
n_total_features = len(mydata.columns) 
n_aleatoric_features = len(['Load','LBMP'])
n_deterministic_features = n_total_features - n_aleatoric_features

# Splitting dataset into train/val/test
training_data = mydata.iloc[:val_time]
validation_data = mydata.iloc[val_time:test_time]
test_data = mydata.iloc[test_time:]


# Now we get training, validation, and test as tf.data.Dataset objects

batch_size = 32

training_windowed = create_dataset(training_data, 
                                   n_deterministic_features,
                                   window_len,
                                   forecast_len,
                                   batch_size)

validation_windowed = create_dataset(validation_data, 
                                     n_deterministic_features,
                                     window_len,
                                     forecast_len,
                                     batch_size)

test_windowed = create_dataset(test_data, 
                               n_deterministic_features,
                               window_len,
                               forecast_len,
                               batch_size=1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


✈ 🏸 stack lstm 

✈ 🏸 latent dim

✈ 🏸 Dense

✈ 🏸 MAPE

✈ 🏸 encoder input & decoder input

In [11]:
latent_dim = 16

# First branch of the net is an lstm which finds an embedding for the past
past_inputs = tf.keras.Input(shape=(window_len,n_total_features), name='past_inputs')
# Encoding the past
encoder = tf.keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(past_inputs)

future_inputs = tf.keras.Input(shape=(forecast_len,n_deterministic_features), name='future_inputs')
# Combining future inputs with recurrent branch output
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True)
x = decoder_lstm(future_inputs, 
                               initial_state=[state_h, state_c])

x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=[past_inputs,future_inputs], outputs=output)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.Huber()
model.compile(loss=loss, optimizer=optimizer, metrics=["mae"])

✈ 🏸 best epoch + early stopping

In [12]:
history = model.fit(training_windowed,epochs=5, validation_data=validation_windowed)

Epoch 1/5
765/765 [==============================] - 17s 14ms/step - loss: 0.0051 - mae: 0.0710 - val_loss: 0.0011 - val_mae: 0.0380
Epoch 2/5
765/765 [==============================] - 13s 14ms/step - loss: 8.8344e-04 - mae: 0.0334 - val_loss: 3.9802e-04 - val_mae: 0.0224
Epoch 3/5
765/765 [==============================] - 12s 13ms/step - loss: 3.7498e-04 - mae: 0.0215 - val_loss: 2.7296e-04 - val_mae: 0.0185
Epoch 4/5
765/765 [==============================] - 14s 14ms/step - loss: 2.1929e-04 - mae: 0.0164 - val_loss: 1.8121e-04 - val_mae: 0.0151
Epoch 5/5
765/765 [==============================] - 13s 13ms/step - loss: 1.7000e-04 - mae: 0.0145 - val_loss: 1.4793e-04 - val_mae: 0.0139


✈ 🏸 plot of loss and val_loss

In [13]:
model.evaluate(test_windowed)

5266/5266 [==============================] - 14s 2ms/step - loss: 1.7119e-04 - mae: 0.0147


[0.000171193532878533, 0.014708106406033039]